In [1]:
import unicodedata, string, time, requests, pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

In [18]:
def get_individual_info(url, key):
    temp = []
    gia_no = []
    agent_name = []

    driver = webdriver.Chrome('./chromedriver.exe')
    driver.get(url)
    driver.find_element_by_xpath('//*[@id="rbl_SearchType_0"]').click()
    driver.find_element_by_xpath('//*[@id="ddl_SearchCriteria"]').send_keys('Agent Name')
    driver.find_element_by_xpath('//*[@id="txt_SearchValue"]').send_keys(key)
    driver.find_element_by_xpath('//*[@id="btnSearch"]').click()
    time.sleep(1.5)
    
    result_html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(markup=result_html, features='html.parser')
    for info in soup.find_all('tr', {'class':['griditem', 'gridalternatingitem']}):
        for details in info.find_all('td'):
            temp.append(details.text.lstrip())
            
    for i in range(0, len(temp), 2):       
        gia_no.append(temp[i])
        agent_name.append(temp[i+1])        
    
    return gia_no, agent_name

def get_agent_details(url, gia_no):
    a_info = []
    n_info = []
    
    url = url+str(gia_no)
    req = requests.Session()
    sess = req.get(url)    
    content = BeautifulSoup(sess.content, 'html.parser')
    
    for info in content.find_all('span', {'id':['AgentInfor_lblGIANoDetails', 'AgentInfor_lblAgentNameDetails', \
                                   'AgentInfor_lblAgentTypeDetails', 'AgentInfor_lblTradeSpecificDetails', 
                                   'PrincipalInfor_lblPrincipalDetails']}):
        if info is None:
            a_info.append('None')
        else:
            a_info.append(unicodedata.normalize('NFKD', info.text.strip()))
    
    for details in content.find_all('tr', {'class':['griditem', 'gridalternatingitem']}):
        for deep in details.find_all('td'):
            if deep is None:
                n_info.append('None')
            else:
                n_info.append(unicodedata.normalize('NFKD', deep.text.strip()))
            
    return a_info, n_info

def check_text(text):
    if text.find('dP'):
        return text.replace('dP', 'd P')        

In [3]:
search_params = list(string.ascii_lowercase)
url = 'https://www.arcm.com.sg/App/UI/Common/SearchAgentInfor.aspx?strApplicationType=SearchNominee'

gia = []
name = []

for alpha in search_params:
    gia_no, a_name = get_individual_info(url, alpha)
    gia.extend(gia_no)
    name.extend(a_name)
    
results = pd.DataFrame(data={'GIA No':gia, 'Agent Name':name})
results.drop_duplicates(inplace=True)

In [4]:
results.to_excel('./Agents.xlsx', index=False)

In [3]:
test_df = pd.read_excel('./Agents.xlsx')

In [4]:
test_df.head(5)

,GIA No,Agent Name
0,A006039,A Tirumavalavan
1,C003300,A W FROZ KHAN NOON
2,C003542,Aaron Kok Kitt Loong
3,C005067,AARON TAN WEI CHENG
4,C004720,Abdul Aziz Bin Syed Sulaiman


In [19]:
test = []

for e in range(10):
    d, n = get_agent_details(url=url, gia_no=test_df['GIA No'][e])
    data = {
        'Agent No':d[0],
        'Agent Name':d[1],
        'Type Of Trade':d[2],
        "Agent's principal":check_text(d[3]),
        'Nominee(s) Information':n
    }
    test.append(data)

In [20]:
pd.DataFrame(test).to_excel('wadad.xlsx', index=False)

In [6]:
url = 'https://www.arcm.com.sg/App/UI/Common/SearchAgentInfoDetails.aspx?GIANo='

In [55]:
x,y = get_agent_details(url=url, gia_no='A004917')

Tan Catherine
Non-Life (General Insurance Agents)
Tan Chek Hwen, Justin
Non-Life (General Insurance Agents)
 ANG HUI PING (HONG HUIPING)
Non-Life (General Insurance Agents)


In [56]:
len(x)

4

In [57]:
x

['A004917',
 'Tan Catherine',
 'General Insurance',
 'Primary Principal:  China Taiping Insurance (Singapore) Pte LtdPayment Type:  Credit']

In [59]:
len(y)

6

In [58]:
y

['Tan Catherine',
 'Non-Life (General Insurance Agents)',
 'Tan Chek Hwen, Justin',
 'Non-Life (General Insurance Agents)',
 'ANG HUI PING (HONG HUIPING)',
 'Non-Life (General Insurance Agents)']